In [1]:
import os
os.chdir('..')

In [2]:
from sklearn.model_selection import train_test_split

import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric

import numpy as np
import torch
from torch.utils.data import Subset
from torch.utils.data import DataLoader

from nn_datasets.bert import BertDataset

In [2]:
bert_pretrained = 'bert-base-uncased'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [ ]:
dataset = BertDataset('lemmatized',
                      bert_pretrained,
                      130,
                      hugging_face = True)

train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size = .4)
test_indices, val_indices = train_test_split(test_indices, test_size = .5)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

train_dataloader = DataLoader(train_dataset, batch_size = 32)
val_dataloader = DataLoader(val_dataset, batch_size = 32)
test_dataloader = DataLoader(test_dataset, batch_size = 32)

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred: transformers.trainer_utils.EvalPrediction) -> dict:
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = BertForSequenceClassification.from_pretrained(bert_pretrained, num_labels = 2)
model.to(device)
training_args = TrainingArguments("bert_trainer", 
                                  num_train_epochs = 2,
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32,
                                  evaluation_strategy = 'steps',
                                  eval_steps = 50,
                                  logging_steps = 50,
                                  )

In [16]:
trainer = Trainer(model=model, 
                  args = training_args, 
                  train_dataset = train_dataset, 
                  eval_dataset = val_dataset, 
                  compute_metrics = compute_metrics)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4567
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 286


Step,Training Loss,Validation Loss,Accuracy
50,0.558700,0.497368,0.765594
100,0.467900,0.455308,0.798424
150,0.422500,0.443309,0.811556
200,0.350100,0.469934,0.793828
250,0.314600,0.455483,0.810243


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=286, training_loss=0.4085308688503879, metrics={'train_runtime': 253.4082, 'train_samples_per_second': 36.045, 'train_steps_per_second': 1.129, 'total_flos': 610201815147600.0, 'train_loss': 0.4085308688503879, 'epoch': 2.0})

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


{'epoch': 2.0,
 'eval_accuracy': 0.8135259356533159,
 'eval_loss': 0.45074746012687683,
 'eval_runtime': 11.881,
 'eval_samples_per_second': 128.188,
 'eval_steps_per_second': 4.04}